In [143]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
from selenium import webdriver
import pandas as pd
import warnings
import random
import timeit
from tqdm import tqdm

warnings.filterwarnings(action='ignore')

In [125]:
# string_data = string_data.split('\n')

In [126]:
# string_data

In [116]:
item_df = pd.read_excel('./상품 목록.xlsx' , index_col='index')

In [139]:
item_df

,name
index,
0,NM2DN51A_빅샷
1,THINK PACK
2,메쉬 스트링 백팩 블랙
3,앱솔루트 백팩 블랙
4,슈퍼브레이크 BLACK
...,...
3151,럭비 맨투맨 멜란지 그레이
3152,레터링 페이스 자수 스Ÿ‡ 셋업
3153,[Fallett x COVERNAT] 코인 아치로고 리버시블 맨투맨 네이비


In [145]:
data_list = item_df['name']
# 3155개 상품까지 리뷰 끌어오기

In [146]:
data_list

index
0                                     NM2DN51A_빅샷
1                                      THINK PACK
2                                    메쉬 스트링 백팩 블랙
3                                      앱솔루트 백팩 블랙
4                                    슈퍼브레이크 BLACK
                          ...                    
3151                               럭비 맨투맨 멜란지 그레이
3152                            레터링 페이스 자수 스Ÿ‡ 셋업
3153    [Fallett x COVERNAT] 코인 아치로고 리버시블 맨투맨 네이비
3154                   [SET] 위켄드 로우 트레이닝 셋업 (오트밀)
3155        [패키지] 유스풀 벌룬 스웨트 세트 SJMT1330+SJLP1328
Name: name, Length: 3156, dtype: object

In [147]:
# 상품명을 쿼리로 검색했는데 안나오는 제품들도 있습니다. => 그 경우 예외 처리하였습니다
# 태그 패턴이 두가지 (하나만 나오는 경우, 여러개가 나오는 경우) 여서 2가지로 예외 처리 하였습니다
# 검색이 안되는 경우는 품절된 경우

item_list = []
error_item_list = []

def get_data_no(data_list):
    
    cnt = 0
    
    for idx, text in tqdm(enumerate(data_list)) :
        try :
            url = f"https://www.musinsa.com/search/musinsa/integration?type=&q={text}"
            r = requests.get(url)
            content = r.content
            soup = BeautifulSoup(content, "html.parser")
            #searchList > li > div.li_inner > div.list_img > a
            try : 
                item_no = soup.select("#searchList > li > div.li_inner > div.list_img > a")[0]["data-bh-content-no"]
            
            except :
                try :
                    item_no = soup.select("#searchList > li:nth-child(1) > div.li_inner > div.list_img > a")[0]["data-bh-content-no"]
                except :
                    print(f'{idx} : {text} -> search error')
                    error_item_list.append(text)
                    cnt +=1
                    continue
                    
            item_dict = dict()
            item_dict["name"] = text
            item_dict["data-no"] = item_no
            item_list.append(item_dict)
            print(f'{idx} : {text}')
            
        except :
            print(f'{idx} : {text} -> search error')
            error_item_list.append(text)
            cnt +=1
            pass
        
    print()    
    print(f'{cnt} # of item search error occured' )

get_data_no(data_list)

2it [00:00,  5.31it/s]

0 : NM2DN51A_빅샷
1 : THINK PACK


3it [00:00,  4.06it/s]

2 : 메쉬 스트링 백팩 블랙


5it [00:01,  4.47it/s]

...

3153 : [Fallett x COVERNAT] 코인 아치로고 리버시블 맨투맨 네이비


3155it [13:05,  3.85it/s]

3154 : [SET] 위켄드 로우 트레이닝 셋업 (오트밀)


3156it [13:05,  4.02it/s]

3155 : [패키지] 유스풀 벌룬 스웨트 세트 SJMT1330+SJLP1328

151 # of item search error occured


In [149]:
error_item_list

['NM2DN53A 보레알레스2',
 '어센틱 원웨이 백팩 블랙',
 '메시 포켓 스트링 짐색 / 그레이',
 'Nylon Slouch Backpack Black',
 '백팩 - 화이트 / H51001',
 'Nylon String Rucksack Blue/Gray',
 '버클 포인트 투웨이 백팩',
 '시그니쳐 카드 홀더_아이보리',
 '리얼 칵투스 카드 홀더-칵투스 그린',
 '공용 인트레치아토 집 어라운드 코인 지갑 - 블랙 / 114075V001N1000',
 '트위치 로고 코듀라 메신저백 블랙',
 'C 로고 투웨이 백 블랙',
 'Nylon Tactical Messenger Bag Black',
 'Cordura Belt bag [BLACK]',
 'Yoko Bucket Bag (M)(cherokee)',
 'capture bag - crinkle black',
 'DP1386 001 블랙 스컬티드 모노그램 폰 크로스바디',
 '[뉴스쿨] 블랙 크로스겸용 쇼퍼백 JLBA2E000BK',
 'FABRIC HOBO BAG - BLACK',
 'UL 1 Mini Bag Black',
 'Liv bag_cloud white',
 '비에티 숄더백 [OH]VIETTI SHOULDER BAG',
 '엘바 미니백 - 레몬',
 'MIDDLE QUILTING BAG IN BLACK',
 'Rowie nylon shoulder bag Ivory',
 '티니백 18 크랙 - 블루그레이',
 'COCSX M61446 IAI INDIGO DANIELA 다니엘라 데님 인디고 에코백',
 '럭키 캔버스 버켓백 3850 아이보리',
 'TYVEK SMALL BAG - BROWN',
 '에코 프렌들리 백 코니 플러스 0043 아이보리',
 '드로스트링 백 퓨어릴리_스카이블루',
 '브라질리아 S 더플백 9.5 (41L) DM3976_010',
 '비슬론 힙색 / 아이보리',
 'UL Neck Pouch Black',
 'OS LOGO PIGMENT BALLCAP_CHARCOAL'

In [150]:
item_list

[{'name': 'NM2DN51A_빅샷', 'data-no': '1034480'},
 {'name': 'THINK PACK', 'data-no': '2066837'},
 {'name': '메쉬 스트링 백팩 블랙', 'data-no': '1264107'},
 {'name': '앱솔루트 백팩 블랙', 'data-no': '693318'},
 {'name': '슈퍼브레이크 BLACK', 'data-no': '1528287'},
 {'name': '어센틱 로고 럭색 33L 블랙', 'data-no': '1749492'},
 {'name': '럭셔리 백팩_Black', 'data-no': '1320752'},
 {'name': 'TWO POCKET BACKPACK', 'data-no': '913402'},
 {'name': '빅스튜던트 BLACK', 'data-no': '1581689'},
 {'name': 'DANKER LIGHT(댄커 라이트) 블랙 (UN123CBP17)', 'data-no': '1263004'},
 {'name': '코듀라 백팩 [블랙]', 'data-no': '2066837'},
 {'name': '슈퍼브레이크 플러스 랩탑 BLACK', 'data-no': '1534526'},
 {'name': '깡스타일리스트 Ⅹ SHOOPEN 백팩 TPBF79W71', 'data-no': '2216497'},
 {'name': '엔보이 BLACK', 'data-no': '1744101'},
 {'name': '코듀라 빅 사이즈 백팩 [블랙]', 'data-no': '1604110'},
 {'name': '스트링 백팩 KS [블랙]', 'data-no': '1261853'},
 {'name': 'NM2DN52A 핫샷', 'data-no': '1034479'},
 {'name': '유니섹스 테크니컬 베를린 백팩 aaa237u', 'data-no': '1323612'},
 {'name': '트위치 로고 유틸리티 럭색 블랙', 'data-no': '2051590'}

In [151]:
# 데이터 프레임 생성
df_item_no = pd.DataFrame(item_list)
df_item_no.to_csv('item_no.csv', index = True)

In [152]:
df_item_no

,name,data-no
0,NM2DN51A_빅샷,1034480
1,THINK PACK,2066837
2,메쉬 스트링 백팩 블랙,1264107
3,앱솔루트 백팩 블랙,693318
4,슈퍼브레이크 BLACK,1528287
...,...,...
3000,[SET] 스테디 스웨트 기모 셋업 2P 오트밀,2234983
3001,럭비 맨투맨 멜란지 그레이,2294685
3002,[Fallett x COVERNAT] 코인 아치로고 리버시블 맨투맨 네이비,2144897
3003,[SET] 위켄드 로우 트레이닝 셋업 (오트밀),2410865


In [123]:
# # 리뷰 끌어오는 코드 테스트 

# num = '1604110'

# url = f'https://www.musinsa.com/app/goods/{num}'

# driver.get(url)

# # 1초에서 2초 사이 랜덤하게 휴식
# driver.implicitly_wait(random.randint(1, 2))

# # 바뀐 url 정보를 parsing해서 soup변수에 저장
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# review_list = []

# for i in range(1,4):
    
#     review_dict = dict()

#     review = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-contents > div.review-contents__text")[0].text
#     positive_feedback = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-evaluation-button > ul > li:nth-child(1) > label > span")[0].text
#     style_feedback = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-evaluation-button > ul > li:nth-child(2) > label > span")[0].text

#     height_test = f"#reviewListFragment > div:nth-child({i}) > div.review-profile > div > div.review-profile__information > p"

#     if not len(soup.select(height_test)) == 0 :
#         height = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-profile > div > div.review-profile__information > p")
#     else:
#         height = None

#     review_dict['review'] = review
#     review_dict['positive_feedback'] = int(positive_feedback)
#     review_dict['style_feedback'] = int(style_feedback)
#     review_dict['height'] = height

#     review_list.append(review_dict)
        
# print(review_list)

#### 상품 크롤링

In [182]:
item_list = pd.read_csv('item_no.csv', index_col='Unnamed: 0')

In [193]:
item_list = item_list.iloc[0:1000]
item_list

,name,data-no
0,NM2DN51A_빅샷,1034480
1,THINK PACK,2066837
2,메쉬 스트링 백팩 블랙,1264107
3,앱솔루트 백팩 블랙,693318
4,슈퍼브레이크 BLACK,1528287
...,...,...
995,MOTORCYCLE B.B CAP RED,2157080
996,Typeservice Web Stitch Cap [Indigo Denim],2507021
997,hide 로고 볼캡(GREEN),2134282
998,빈티지 캐쥬얼 볼캡 빈티지 그레이,2560668


In [192]:
for idx, row in item_list.iterrows():
    print(row['data-no'])

1034480
2066837
1264107
693318
1528287
1749492
1320752
913402
1581689
1263004
2066837
1534526
2216497
1744101
1604110
1261853
1034479
1323612
2051590
945749
472915
1986031
472909
747317
926836
1466741
1534520
2231913
2306062
1753488
1090160
1264107
1255947
2306064
2051590
1276702
1727859
1446582
713130
1492969
1220315
933160
1417382
1973470
1744100
1776606
1748195
2231912
1564324
1749482
931548
2355716
2193458
1188993
2071229
693329
1540979
1802030
1704593
1753490
1234658
1597319
2286647
1090158
1208468
1838014
2066838
1527984
1090159
2315011
1727788
472907
1703732
933761
1255942
2066004
388328
1688205
2279619
1740940
1507125
2308901
1895457
2354874
2244919
2126904
1528044
2042924
2415094
937922
2271798
2370401
963672
1773180
2295293
2163965
1264107
2030465
473839
571593
1745100
2367532
2283502
839841
1757159
165155
2279622
2215256
2311383
2306065
1895450
2557231
1736025
2229665
1726518
950226
2073103
1726504
2370400
1745413
1906943
1878051
2014459
1059077
2193458
1865993
1865963
16840

In [196]:
# review_list는 지역 변수
# review_dict는 지역 변수 => review dict 객체를 review_list에 추가하여 Json 형태로 만든다.

def crawling(item_list):
    cnt = 0
    
    driver = webdriver.Chrome("/Users/yoohajun/Library/Mobile Documents/com~apple~CloudDocs/Hajun/프로젝트/Cordi_RecSys/crawling/chromedriver")
    driver.implicitly_wait(3)
    
    review_list = []
    
    # item
    for idx, row in tqdm(item_list.iterrows()) :
        try :
        
            num = row['data-no']

            url = f'https://www.musinsa.com/app/goods/{num}'

            driver.get(url)

            # 1초에서 2초 사이 랜덤하게 휴식
            driver.implicitly_wait(random.randint(1, 2))

            # 바뀐 url 정보를 parsing해서 soup변수에 저장
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 추천 순으로 끌어오되 실패하면 그대로 최신순으로 끌어오기
            try : 
                driver.find_element_by_css_selector("#reviewSelectSort > option:nth-child(3)").click();
            except :
                pass


            # 상세페이지 아이템 끌어오기 3개의 리뷰
            for i in range(1,4):

                # 모든 리뷰마다 만약 웹구조가 바뀌어서 크롤링 에러가 뜰 경우 해당 리뷰는 예외 처리
                try :
                    review_dict = dict()

                    review = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-contents > div.review-contents__text")[0].text
                    positive_feedback = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-evaluation-button > ul > li:nth-child(1) > label > span")[0].text
                    style_feedback = soup.select(f"#reviewListFragment > div:nth-child({i}) > div.review-evaluation-button > ul > li:nth-child(2) > label > span")[0].text

                    height_test = f"#reviewListFragment > div:nth-child({i}) > div.review-profile > div > div.review-profile__information > p"

                    if not len(soup.select(height_test)) == 0 :
                        height_temp = soup.select(f"#reviewListFragment > div:nth-child(1) > div.review-profile > div > div.review-profile__information > p")[0].text
                        height = [elem.strip() for elem in height_temp.split(',')]
                    else:
                        height = None

                    review_dict['data-no'] = row['data-no']
                    review_dict['name'] = row['name']
                    review_dict['review'] = review
                    review_dict['positive_feedback'] = int(positive_feedback)
                    review_dict['style_feedback'] = int(style_feedback)
                    review_dict['height'] = height

                    # 끌어온 리뷰 리스트에 추가
                    review_list.append(review_dict)
                    print(f'# {idx}, {review_dict}')

                except : 
                    print(f'# {num} : {i}, review failed' )
                    cnt += 1
                    pass

                # 만약 컴퓨터에 부하가 많이 걸리면 아래 프린트 문 없애도 됩니다 => 디버깅 용
                #print(review_list)

        except : continue

    print(f'{cnt}번 리뷰 크롤링 실패')
    return review_list

In [ ]:
review_list = crawling(item_list)

0it [00:00, ?it/s]

# 0, {'data-no': 1034480, 'name': 'NM2DN51A_빅샷', 'review': '이름 그대로 되게 큰 백팩인데 힙해서 맘에들어요 수납공간도 많고 노스페이스라 그런지 생활방수 정도도 되는거 같아요', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}
# 0, {'data-no': 1034480, 'name': 'NM2DN51A_빅샷', 'review': '확실히 빅샷 답게 수납공간도 넉넉하며 힙한 누낌까지 아주 잘 이용하고 있습니다!!', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}


1it [00:13, 13.32s/it]

# 0, {'data-no': 1034480, 'name': 'NM2DN51A_빅샷', 'review': '시티보이룩에 딱 적합한 옷이네요. 그런데 생각보다 커서 손은 별로 안갈거같아요', 'positive_feedback': 1, 'style_feedback': 0, 'height': None}
# 1, {'data-no': 2066837, 'name': 'THINK PACK', 'review': '공간이 넓어서 큰 책 여러 개도 잘 넣고 다닐 수 있어요', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}
# 1, {'data-no': 2066837, 'name': 'THINK PACK', 'review': '정말 마음에 드네요 예쁘게 잘 쓰고 있습니다.크기도 딱 적당해요', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}


2it [00:18,  8.60s/it]

...

# 295, {'data-no': 1556574, 'name': 'JOELLE FALLIN SM 크로스 BK (JHNCHA0BF830BK010)', 'review': '캐주얼 정장 모두 잘 어울립니다 .금속 부분이 처음에 봤을 땐 골드였는데 몇달 지나니까 색상이 변색 되는 것 같아요 .가격대비 퀄리티 추천 드리지는 않고요 디자인때문이라면 앙증맞고 귀여운 것 같아요 . 잠금장치가 돌리는 스타일이라 조금 불편합니다 .사이즈 생각보다 작 았구요 반지갑이나 휴대폰 화장품 정도 들어갈 수 있는 사이즈에요 가죽인지 합피인지 정확하게 모르겠지만 재질이 고급스럽지는않습니다 ', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}
# 296, {'data-no': 1366525, 'name': 'FABRIC SLING BAG - WHITE', 'review': '사이즈도 적당하고 재질은 좀 구리지만 괜찮에용', 'positive_feedback': 0, 'style_feedback': 1, 'height': None}
# 296, {'data-no': 1366525, 'name': 'FABRIC SLING BAG - WHITE', 'review': '진짜 휘뚜루마뚜루 최고.. 매장에서 보고 반해서 바로 구매 갈겼습니다.. 여름에 딱 매기 좋아요! 아주 잘드는중', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}


297it [17:16,  2.97s/it]

# 296, {'data-no': 1366525, 'name': 'FABRIC SLING BAG - WHITE', 'review': '어떤옷에도 무난하게 잘 어울리고 포인트가 되는것 같아요 다만 물건을 많이 넣을순 있지만 예쁘게 들려면 적당히 넣는걸로... 그리고 포켓은 너무 잘산거 같아요 ', 'positive_feedback': 1, 'style_feedback': 0, 'height': None}
# 297, {'data-no': 472917, 'name': 'D89 메신져 크로스백(블랙)', 'review': '믿고 쓰는 칼하트! 생각보다 조금 크긴한데 많이 들어가면 좋죠', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}
# 297, {'data-no': 472917, 'name': 'D89 메신져 크로스백(블랙)', 'review': '칼라트 메신저백 이쁘네요 놀러가서 잘 썼습니다~', 'positive_feedback': 0, 'style_feedback': 1, 'height': None}


298it [17:18,  2.86s/it]

# 297, {'data-no': 472917, 'name': 'D89 메신져 크로스백(블랙)', 'review': '수납공간도 생각보다 넓은데 앞쪽 주머니가 작은게 조금 아쉬움', 'positive_feedback': 0, 'style_feedback': 0, 'height': None}


In [179]:
# 데이터 프레임 생성
df = pd.DataFrame(review_list)

In [180]:
df 

,data-no,name,review,positive_feedback,style_feedback,height
0,1581689,빅스튜던트 BLACK,수납이 넉넉해서 여행갈때 짐 걱정 없을거같아요 잘 쓰겠습니당,0,0,None
1,1581689,빅스튜던트 BLACK,"진짜,, 진짜 엄청 커져요! ㅋㅋ 수납력 대박입니다부산 1박2일 여행 이거 들고 갔...",1,0,None
2,1581689,빅스튜던트 BLACK,아직 책이나 짐은 안넣어봤지만 사람들이 많이 사는 이유가 있겠죠,1,6,None
3,1263004,DANKER LIGHT(댄커 라이트) 블랙 (UN123CBP17),생각보다 크지만 큰 가방을 찾고 있었어서 만족합니다 디자인도 예쁘고 오래오래 쓸거같...,0,0,None
4,1263004,DANKER LIGHT(댄커 라이트) 블랙 (UN123CBP17),남자친구 생일 선물로 줬는데 너무 좋아해요😆수납 공간도 넉넉하구 스타일도 이쁘고~ ...,0,1,None
...,...,...,...,...,...,...
117,1396732,서핀 백 프린트 티 셔츠_써니 옐로우,예뻐요 색깔도 쨍해서 여름에 잘 입고 다닐거 같아요,0,0,None
118,1396732,서핀 백 프린트 티 셔츠_써니 옐로우,딱 제가 원하던 오버핏입니다어떤옷이든 입기 좋아요,0,0,None
119,2472395,ATH TEE(SAINT BASEBALL)_NAVY,색상 너무 이뻐요!! 사이즈도 적당합니다 어디에나 잘 어울려요,0,0,"[남성, 176cm, 75kg]"
120,2472395,ATH TEE(SAINT BASEBALL)_NAVY,티셔츠 색도 그렇고 프린팅도 그렇고 너무 이뻐요,0,0,None


In [ ]:
# 신장 정보가 많이 없는 것 같습니다
df.height.isnull().sum()

In [ ]:
# csv 저장
df.to_csv('0_999번_아이템리뷰.csv', index=True)

In [ ]:
driver.quit()